In [1]:
import psycopg2
import wget
import sqlite3

In [2]:
# define user, dbname, password, and host (from ElephantSQL)

dbname = 'ceforqty'

user = 'ceforqty'

password = 'GYh0O6sCjsdPPgZXg4DRYzL3Y9LDC5gl'

host = 'salt.db.elephantsql.com'

In [20]:
# establish connection to elephantsql
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [21]:
# create cursor
pg_curs = pg_conn.cursor()

In [22]:
# pull RPG data from lambda github repo
wget.download('https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true')

'rpg_db (7).sqlite3'

In [6]:
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [7]:
# create count variable
count = sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

In [8]:
# extract character list
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [9]:
# check dtypes of character table
sl_curs.execute('PRAGMA table_info(charactercreator_character);').fetchall()

[(0, 'character_id', 'integer', 1, None, 1),
 (1, 'name', 'varchar(30)', 1, None, 0),
 (2, 'level', 'integer', 1, None, 0),
 (3, 'exp', 'integer', 1, None, 0),
 (4, 'hp', 'integer', 1, None, 0),
 (5, 'strength', 'integer', 1, None, 0),
 (6, 'intelligence', 'integer', 1, None, 0),
 (7, 'dexterity', 'integer', 1, None, 0),
 (8, 'wisdom', 'integer', 1, None, 0)]

In [10]:
# Extract done! Next step - transform
# We need the PostgreSQL db to have a table with
# an appropriate schema
create_character_table = """
  CREATE TABLE charactercreator_character (
    character_id SERIAL PRIMARY KEY,
    name VARCHAR(30),
    level INT,
    exp INT,
    hp INT,
    strength INT,
    intelligence INT,
    dexterity INT,
    wisdom INT
  );
"""

In [11]:
# execute creation of table
pg_curs.execute(create_character_table)

In [12]:
# We can query tables if we want to check
# This is a clever optional thing, showing postgresql internals
show_tables = """
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
"""
pg_curs.execute(show_tables)
pg_curs.fetchall()

[('public',
  'charactercreator_character',
  'ceforqty',
  None,
  True,
  False,
  False,
  False)]

In [13]:
# create sample insert
example_insert = """
INSERT INTO charactercreator_character
(name, level, exp, hp, strength, intelligence, dexterity, wisdom)
VALUES """ + str(characters[0][1:])

In [14]:
example_insert

"\nINSERT INTO charactercreator_character\n(name, level, exp, hp, strength, intelligence, dexterity, wisdom)\nVALUES ('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [15]:
# How do we do this for all characters? Loops!
for character in characters:
  insert_character = """
    INSERT INTO charactercreator_character
    (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
    VALUES """ + str(character[1:]) + ';'
# print(insert_character)
pg_curs.execute(insert_character)

In [16]:
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_curs.fetchall()

[(1, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)]

In [17]:
# OK, we can see it in elephantsql.com now
# But let's also check programmatically
pg_curs = pg_conn.cursor()  # Make a new cursor
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [19]:
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character

AssertionError: 